In [1]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276

In [2]:
import numpy as np
import os
#import sox
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
import matplotlib.pyplot as plt
#import seaborn as sn
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew, entropy
from scipy.signal import lfilter
import librosa
import math
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix

## Settings

In [3]:
##DATASET

#CHANGE TO YOUR OWN TRAINING SET FOLDER
#folder of training data set
FOLDER_PATH = 'data/YT_set/edited_wavs/'

#CHANGE TO YOUR OWN FOLDER TO STORE NORMALIZED WAVS
#folder where normalized wavs are stored
NORM_FOLDER_PATH = 'data/YT_set/edited_wavs/norm/'

norm_skip = True #skip normalization step


##FEATUTRES

featExtr_skip = False

#Initialize data frame of features:

feats = pd.DataFrame([])

#tiny constant value
eps = sys.float_info.epsilon

#Features' settings:

fs_targ = 16000 # set all audios to this sampling frequency
n_channels_targ = 1

#framing
frame_len_s=0.025 #12 segments seemed adequeate in paper, since segments are no longer than 400ms (400ms/12=33.3ms)
frame_step_s=frame_len_s #according to paper: non-overlapping frames

frame_len = int(round(frame_len_s*fs_targ)) #in samples
frame_step = int(round(frame_step_s*fs_targ)) #in samples
win_func =np.hamming #at least for mfcc

#mfcc
cep_num= 13 #number of coefficients as in paper (https://link.springer.com/article/10.1007/s10439-013-0741-6)

#lp
lp_ord = int(round(2 + fs_targ/1000)) #standard rule of thumb for LP oder

#formants
nr_formants = 4 #as in paper, first 4 formants

## Functions

In [56]:
#Apply pre-emphasis (high-pass) filter
def apply_preEmph(x):
    x_filt = lfilter([1., -0.97], 1, x)
    return x_filt
        
#Obtain autocorrelation
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[int((result.size+1)/2):] #Note: other people use re.size/2:, but this does not work for me 
                                   # TODO: check consistency in other computers

#Compute zero-crossing rate
def get_zcr(x):
    zcr = (((x[:-1] * x[1:]) < 0).sum())/(len(x)-1)
    return zcr

#Compute log-energy
def get_logEnergy(x):
    logEnergy = np.log10( ( (np.power(x,2)).sum()/len(x) ) + eps)  
    return logEnergy

#Estimate fundamental frequency (F0)
def get_F0(x,fs):
    #autocorrelation-based method to extract F0
    xcorr_arr = autocorr(x)
    
    #looking for F0 in the frequency interval 50-500Hz, but we search in time domain
    min_ms = round(fs/500)
    max_ms = round(fs/50)
    
    xcorr_slot = xcorr_arr[max_ms+1:2*max_ms+1]
    xcorr_slot = xcorr_slot[min_ms:max_ms]
    t0 = np.argmax(xcorr_slot)
    F0 = fs/(min_ms+t0-1)
    return F0

#Estimate formants
def get_formants(x, lp_order, nr_formants):
    
    #compute lp coefficients
    a = librosa.lpc(x, lp_ord)
    

    #get roots from lp coefficients
    rts = np.roots(a)
    rts = [r for r in rts if np.imag(r) >= 0]

    #get angles
    angz = np.arctan2(np.imag(rts), np.real(rts))

    #get formant frequencies
    formants = sorted(angz * (fs_targ / (2 * math.pi)))
    
    return formants[0:nr_formants]

#Extract frequencies
def feature_extraction(x,fs,feats_df,lp_ord,ID,label):
#Extract features from signal x (identified as ID), and concatenate them to dataframe feats_df
#Features' reference: (see Appendix)
#[1]https://link.springer.com/article/10.1007/s10439-013-0741-6
#[2]https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?dsi_version=c5434db897ab74b192ca295a9eeca041&Expires=1585086202&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=c8k8DmG~KIxg0ToTO8rebm2MzHneCzJGkjSFRB7BYTEQ-MHXEr0ocHmISrldP3hFf9qmeiL11ezyefcNeRVeKIQ9PVjOl9pn7rXWcjA1o2voPn1VnDd8n7G2cT31apdj0LNMclhlXRPnCsGD66qDRqa3d-xaqqXhEqU73aw3ZgBgroO213MfJOqFhJxxXo2QEia0bSlDRTeX9KhSczFK-IFTPC6GwFL2L04por8pQRI3HF7E3f26O9zp9OhkwxSU9qfJah20WxZLA4PxREdv7JGoVBinR6T0mTcIaQi~B4IzYjSPSsTTADMNk5znVYIvSqgtMT~DY~qwlfq4SRdFjQ__
  
    
    #do features in a frame-basis
    x_frames = spe_feats.sigproc.framesig(x,frame_len,frame_step,win_func) #DOUBT: should I use window or not?
                                                                        #at least for formant estimation i should

    nr_frames = x_frames.shape[0]
    #print(nr_frames)
        
    #0)Wavelets #TODO
    
    #DOUBT: if log-energy feature is included, should I also include the first mfcc coefficient (c0) ?
    #1)mfcc
    mfcc_feat = spe_feats.mfcc(x,fs, winlen=frame_len_s,winstep=frame_step_s, numcep=cep_num,winfunc=win_func)
    
    #deltas to capture 
    mfcc_delta_feat = spe_feats.delta(mfcc_feat,1) #mfcc_delta_feat = np.subtract(mfcc_feat[:-1], mfcc_feat[1:]) #same
    mfcc_deltadelta_feat = spe_feats.delta(mfcc_delta_feat,1)          
    
    #2)zero-crossing rate
    zcr_feat = np.apply_along_axis(get_zcr, 1, x_frames)
    
    #3)Formant frequencies
    #using LP-coeffcs-based method
    #formant_feat = np.apply_along_axis(get_formants, 1, x_frames, lp_ord, nr_formants)
    
    #Note: for the moment, it seems some frames are ill-conditioned for lp computing,
    #current solution - we skip those and fill with NaN values
    formants_feat= np.empty((nr_frames,4))
    formants_feat[:] = np.nan
    
    for i_frame in range(0,nr_frames):
        try: 
            formants_feat[i_frame] = get_formants(x_frames[i_frame], lp_ord, nr_formants)
        except:
            pass
    
    #4)Log-energy
    logEnergy_feat =  np.apply_along_axis(get_logEnergy, 1, x_frames)
    
    #5)Pitch (F0)
    F0_feat =  np.apply_along_axis(get_F0, 1, x_frames,fs)
    
    #TODO: compute also F0 with pysptk (a python wrapper for SPTK library), it probably gives better results
    #https://github.com/r9y9/pysptk/blob/master
    
    #6)Kurtosis
    kurt_feat =  np.apply_along_axis(kurtosis, 1, x_frames)
    
    #7)Bispectrum Score (BGS)
    #TODO: see PhD thesis for more info on this feature
    
    #8)Non-Gaussianity Score (NGS)
    #TODO: see PhD thesis for more info on this feature
   
    #9) Adding skewness as measure of non-gaussianity (not in paper)
    skew_feat =  np.apply_along_axis(skew, 1, x_frames)
    
    #DOUBT: 10) Shannon entropy GETTING -inf in all cases, WHY??? Don't include until fixed
    #entropy_feat = entropy(x)
    #Maybe compute directly to check
    
    mfcc_cols = ['mfcc_%s' % s for s in range(0,cep_num)]
    mfcc_delta_cols = ['mfcc_d%s' % s for s in range(0,cep_num)]
    mfcc_deltadelta_cols = ['mfcc_dd%s' % s for s in range(0,cep_num)]
    formants_cols = ['F%s' % s for s in range(1,nr_formants+1)]
          
    feats_segment = pd.concat([pd.DataFrame({'Id': ID, 'kurt': kurt_feat, 'logEnergy': logEnergy_feat,
                                                 'zcr': zcr_feat, 'F0': F0_feat,
                                                 'skewness': skew_feat, 'label': label}),
                               pd.DataFrame(mfcc_feat,columns=mfcc_cols), 
                            pd.DataFrame(formants_feat,columns=formants_cols)],axis=1)
    
    print(nr_frames)
    feats_df = feats_df.append(feats_segment,ignore_index=True, sort=False)
    
    return feats_df


# MAIN

## Reading recordings + feature extraction

In [82]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [83]:
#Read wav data set, apply pre-processing and extract features

if featExtr_skip is False:

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(FOLDER_PATH) if os.path.isfile(os.path.join(FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = FOLDER_PATH+file_name
        #print(full_fname)
    
        if norm_skip is False: 
        ## Normalization
        
            #name for normalization
            norm_fname = NORM_FOLDER_PATH + os.path.splitext(file_name)[0] + '_NORM.wav'
        
            #level to same dB
            tfm = sox.Transformer()
            tfm.gain(gain_db=0.0, normalize=False, limiter=False, balance=None)
            #downsample to 16kHz and 1 channel
            tfm.convert(samplerate=fs_targ, n_channels=n_channels_targ, bitdepth=None) 
            #tfm.norm(db_level=0.0)
    
            # create the output normalized audio
        
            print(norm_fname)
            tfm.build(full_fname, norm_fname)
            tfm.effects_log
    
            wav_fname = norm_fname
        else:
            wav_fname = full_fname
    
        # load normalized audio
        s = AudioSegment.from_wav(wav_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(wav_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        #print(label)
        all_label.append(label)

i wanted to listen to some:

In [37]:
np.where(np.array(all_label)=='Dry')

(array([ 1,  2,  3,  8, 11, 12, 13, 16, 19, 22, 25, 26, 31, 33, 35]),)

In [36]:
np.where(np.array(all_label)=='Wet')

(array([ 0,  4,  5,  6,  7,  9, 10, 14, 15, 17, 18, 20, 21, 23, 24, 27, 28,
        29, 30, 32, 34]),)

In [42]:
s=all_s[15]
s

resample:

In [86]:
all_s=[s.set_frame_rate(fs_targ) for s in all_s]

In [89]:
fs=fs_targ

feature calculation:

In [90]:
for s, id, label in zip(all_s,all_id,all_label):

        ## Segmentation of cough streams (silence-based)
        #min_silence_len in ms, silence_thresh in dB
        s_segments = split_on_silence (s, min_silence_len = 600, silence_thresh = -30)
        ## TODO: set more accurate thresholds, or find other way to split (variance-based?)
    
        #checks that segmentation and removal of silence is OK
        #print(len(s_segments))
        #for i in range(len(s_segments)):
        #    play(s_segments[i])
        #    input("Press Enter to continue...")
            
    
        #convert s_segments to numpy array format
        AudioSegment2numpy_arr = lambda x: np.asarray(x.get_array_of_samples())
        s_segments_np = list(map(AudioSegment2numpy_arr, s_segments))
    
        print('High-pass filtering...')
        #pre-emphasis filtering to each segment
        preEmph_filtering = lambda x: apply_preEmph(x)
        s_segments_filt = list(map(preEmph_filtering, s_segments_np))
    
        print('Computing features...')
        #Feature extraction for each segment
    
        #(lambda function doesn't work )
        #feat_extr_step = lambda x, fs, feats_df, lp_ord, ID: feature_extraction(x,fs,feats_df,lp_ord,ID)
        #feats = feat_extr_step(s_segments_filt,fs,feats,lp_ord,ID)
        for idx, seg_i in enumerate(s_segments_filt):
            print('\tSegment %d' % idx)
            feats = feature_extraction(seg_i,fs,feats,lp_ord,ID,label)
    
       

High-pass filtering...
Computing features...
	Segment 0
131
131
High-pass filtering...
Computing features...
	Segment 0
88
88
High-pass filtering...
Computing features...
	Segment 0
181
181
High-pass filtering...
Computing features...
	Segment 0
120
120
High-pass filtering...
Computing features...
	Segment 0
130
130
High-pass filtering...
Computing features...
	Segment 0
80
80
	Segment 1
21
21
High-pass filtering...
Computing features...
	Segment 0
81
81
High-pass filtering...
Computing features...
	Segment 0
114
114
High-pass filtering...
Computing features...
	Segment 0
85
85
High-pass filtering...
Computing features...
	Segment 0
84
84
High-pass filtering...
Computing features...
	Segment 0
72
72
	Segment 1
73
73
High-pass filtering...
Computing features...
	Segment 0
168
168
High-pass filtering...
Computing features...
	Segment 0
163
163
High-pass filtering...
Computing features...
	Segment 0
38
38
High-pass filtering...
Computing features...
	Segment 0
111
111
High-pass filtering.

## Load  (or store) features 

In [ ]:

feats_fname = 'feats_df.pkl'

if featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [92]:
feats

,Id,kurt,logEnergy,zcr,F0,skewness,label,mfcc_0,mfcc_1,mfcc_2,...,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,F1,F2,F3,F4
0,Xe68,2.224569,1.950116,0.739348,347.826087,0.063148,Wet,10.946505,-55.622223,-7.902362,...,-12.506665,-1.898333,-6.000742,-11.526866,-20.011184,-1.172862,465.663922,1697.393054,2677.797426,3102.235157
1,Xe68,1.902177,1.925601,0.714286,500.000000,-0.140255,Wet,10.835985,-56.009206,-7.769080,...,-12.528566,-2.273709,2.371082,0.904606,-16.422882,-10.121078,599.794882,1540.778999,2631.118103,3501.133679
2,Xe68,2.336478,1.875439,0.676692,410.256410,0.225282,Wet,10.740739,-56.885048,-6.033646,...,-15.179400,4.140030,-3.388340,0.053187,-4.547879,-10.094373,0.000000,1478.625206,2271.608970,3255.717792
3,Xe68,2.298217,1.912341,0.741855,484.848485,-0.003853,Wet,10.861177,-59.472622,-8.577299,...,-13.946394,-3.915269,-2.598803,-6.396431,-16.800743,1.040624,0.000000,1630.580010,1737.128667,2822.935462
4,Xe68,1.971004,1.798398,0.641604,516.129032,0.083734,Wet,10.481100,-56.990164,-7.697026,...,-11.829854,-4.321207,-3.156944,-14.316977,-14.828545,-3.897658,0.000000,0.000000,1544.967225,2611.943517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4077,Xe68,1.325702,5.627520,0.796992,253.968254,-0.081824,Dry,19.503615,-51.708540,0.836801,...,-2.374808,-19.541898,-3.771205,5.669418,-16.882422,3.156540,0.000000,1009.838176,2298.910316,2715.675572
4078,Xe68,1.256732,5.744825,0.741855,421.052632,0.015154,Dry,19.720318,-54.176969,1.706384,...,-0.188686,-3.103108,-15.566504,-23.650377,-1.996956,1.367044,0.000000,830.499270,1863.007683,2615.245306
4079,Xe68,2.135804,6.029200,0.779449,516.129032,0.309676,Dry,20.394354,-53.465634,2.325907,...,13.628131,-4.982539,-14.584632,-9.345487,-13.468764,9.687079,0.000000,836.544781,2238.814824,2498.094764
4080,Xe68,1.901492,6.083249,0.791980,444.444444,-0.104613,Dry,20.532528,-57.239071,3.652515,...,-0.584251,6.867298,-10.764907,-13.887178,-11.299307,-0.519286,0.000000,847.685585,2132.353112,2570.195569


In [ ]:
#1.Check which columns have NaNs values

#feats2 = feats.copy()

#sum(feats.isna().any())
#feats.columns[feats.isna().any()].tolist() --> We get just the ones we have inserted in formants
feats2 = feats.interpolate(method ='cubic')
#feats2 = feats.dropna(axis=0).copy()
#feats2.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)

#feats2.columns[feats2.isna().any()].tolist()
#feats2.describe()


In [ ]:
sum(feats2.isna().any())

In [ ]:
#Grouping the frames from a same recording (Id) into chunks with the same number of frames.
#The training of the classifier will be based on these chunks

feats2['cum_IDidx'] = feats2.groupby('Id').cumcount()

def get_subidx(cum_Idx,batch_size):
    #batch needs to be an integer (or float like 3.0)
    return int(1.0*cum_Idx/batch_size)

feats2['subIdx'] = feats2.apply(lambda x: get_subidx(x['cum_IDidx'], 10), axis=1)
feats2 = feats2.drop(['cum_IDidx'],axis=1)

In [ ]:
feats2

In [ ]:
mean_feats = feats2.groupby(['Id','subIdx']).aggregate('mean').reset_index()
std_feats = feats2.groupby(['Id','subIdx']).agg(lambda x: x.std(ddof=0)).reset_index() #ddof=0 to compute population std (rather than sample std)
keep_same = {'Id', 'subIdx'}
mean_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_m') for c in mean_feats.columns]
std_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_std') for c in std_feats.columns]

In [ ]:
sum(std_feats.isna().any())

In [ ]:
mean_std_feats = pd.merge(mean_feats, std_feats, on=['Id','subIdx'], how='outer')

In [ ]:
#Make dictionary and add label column using it 
feats_unique = feats.drop_duplicates(subset=['Id'])
label_dict = dict(zip(feats_unique.Id, feats_unique.label))
mean_std_feats['label'] = mean_std_feats["Id"].map(label_dict)
#mean_std_feats[['Id','label']].head(50)

In [ ]:
mean_std_feats.columns

In [ ]:
mean_std_feats.describe()

In [ ]:
mean_std_feats.columns[mean_std_feats.isna().any()].tolist() 

In [ ]:
mean_std_feats['Id']

In [ ]:
sum(mean_std_feats.isna().any())

In [ ]:
#mean_std_feats = mean_std_feats.interpolate(method ='cubic')
#mean_std_feats.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True) #--> Doesn't work!? Still get NaN error

In [ ]:
sum(mean_std_feats.isna().any())

In [ ]:
mean_std_feats['Id']

In [ ]:
#2. Get feature set, labels, and recording IDs
X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']
ID_list = ID_train.drop_duplicates()

#ID_train.size
ID_list.size

In [ ]:
#3. Normalization in case some model requires it

scaler = StandardScaler()
scaler.fit(X_train)

#use same scaler for both, based on X_train data
X_trainNorm = scaler.transform(X_train.values)

In [ ]:
sum(X_train.isna().any())

## Model training

### Train-test split (k-fold)

In [ ]:
k = ID_list.values.size #number of folds

group_kfold = GroupKFold(n_splits=k)
group_kfold.get_n_splits(X_trainNorm, y_train, ID_train)

### Logistic regression

In [ ]:
#Do cross-validation
pred_probs = pd.DataFrame([])

idx_acc = 0
for train_index, test_index in group_kfold.split(X_trainNorm,y_train,ID_train):
    X_train1, X_test1 = X_trainNorm[train_index], X_trainNorm[test_index]
    y_train1, y_test1 = y_train[train_index], y_train[test_index]
    
    #logReg = LogisticRegression()
    
    #loss=log --> logistic regression
    logReg = SGDClassifier(loss='log', penalty='l2')
    logReg.fit(X_train1, y_train1)
    y_hat_prob = logReg.predict_proba(X_test1)
    classes =logReg.classes_
    pred_probs = pred_probs.append(pd.DataFrame({'ID': ID_train[test_index], str(classes[0]): y_hat_prob[:,0], str(classes[1]): y_hat_prob[:,1]}),ignore_index=True, sort=False)    

In [ ]:
#mean_feats = feats2.groupby(['Id','subIdx']).aggregate('mean').reset_index()
mean_pred_probs = pred_probs.groupby('ID').aggregate('mean').reset_index()

In [ ]:
mean_pred_probs

In [ ]:
def predict_class(prob_dry,prob_wet):
    if prob_dry > prob_wet :
        return 'Dry'
    else:
        return 'Wet'
    
mean_pred_probs['pred_class'] = mean_pred_probs.apply(lambda x: predict_class(x['Dry'], x['Wet']), axis=1)

In [ ]:
#add actual classes
mean_pred_probs['label'] = mean_pred_probs["ID"].map(label_dict)

In [ ]:
mean_pred_probs

In [ ]:
mean_pred_probs[(mean_pred_probs.pred_class != mean_pred_probs.label)]

## Evaluation

In [ ]:
#Accuracy
acc = accuracy_score(mean_pred_probs['label'], mean_pred_probs['pred_class'])
print(acc)

In [ ]:
#TODO: Check if following measures are computed OK

In [ ]:
#Precision
prec = precision_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(prec)

In [ ]:
#F1-score
f1 = f1_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(f1)

In [ ]:
#recall
recall = recall_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(recall)

In [ ]:
#confusion matrix
conf_mat_df = pd.crosstab(mean_pred_probs['label'], mean_pred_probs['pred_class'], margins=True)

In [ ]:
conf_mat_df

In [ ]:
#Cough sound
#Breathing rate
#Breathing rhytm (consistence smoothness)
#Cough rate
#Panic level
#Hoarseness